In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution

# General imports
import os
import json
import numpy as np
from datetime import datetime

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian, create_plots

In [70]:
potential = 'QHO'
cut_off = 2
tol = 1e-3

In [71]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cut_off, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [72]:
# Device
shots = None
dev = qml.device('lightning.qubit', wires=num_qubits, shots=shots)

#Initial params shape
num_layers = 1
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
        

In [88]:
ground_params = pnp.array([4.44063813, 3.14159627, 3.26900029, 2.23356159, 4.46089481,
            2.55241113])
def generate_ground_state(ground_params):
    p = pnp.tensor(ground_params.reshape(params_shape), requires_grad=True)
    qml.StronglyEntanglingLayers(weights=p, wires=range(num_qubits), imprimitive=qml.CZ)

In [89]:
dev = qml.device("default.qubit")

@qml.qnode(dev)
def circuit():
    generate_ground_state(ground_params)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

print(f"Ground state energy: {circuit()}")

Ground state energy: 3.269605695831819e-12


In [74]:
def ansatz(params):
    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    qml.StronglyEntanglingLayers(weights=params, wires=range(num_qubits), imprimitive=qml.CZ)

In [75]:
random_params = np.random.uniform(low=0, high=2 * np.pi, size=np.prod(params_shape)).reshape(params_shape)
print(qml.draw(ansatz)(random_params))

0: ─╭StronglyEntanglingLayers(M0)─┤  
1: ─╰StronglyEntanglingLayers(M0)─┤  
M0 = 
[[[2.51942231 4.11044856 1.08469854]
  [3.22265354 2.02953975 2.67893565]]]


In [91]:
@qml.qnode(dev)
def swap_test(params):

    generate_ground_state(ground_params)
    ansatz(params)

    qml.Barrier()  # added to better visualise the circuit
    qml.Hadamard(wires=0)
    for i in range(num_qubits):
        qml.CSWAP(wires=[0, 1 + i + num_qubits, 1 + i])
    qml.Hadamard(wires=0)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

print(qml.draw(swap_test)(random_params))
print(f"\nOverlap between the ground state and the ansatz: {swap_test(random_params)}")

0: ─╭StronglyEntanglingLayers(M0)─╭StronglyEntanglingLayers(M1)──||──H─╭●────╭●─────H─┤ ╭<𝓗(M2)>
1: ─╰StronglyEntanglingLayers(M0)─╰StronglyEntanglingLayers(M1)──||────├SWAP─│────────┤ ╰<𝓗(M2)>
2: ──────────────────────────────────────────────────────────────||────│─────├SWAP────┤         
3: ──────────────────────────────────────────────────────────────||────╰SWAP─│────────┤         
4: ──────────────────────────────────────────────────────────────||──────────╰SWAP────┤         

M0 = 
[[[4.44063813 3.14159627 3.26900029]
  [2.23356159 4.46089481 2.55241113]]]
M1 = 
[[[2.51942231 4.11044856 1.08469854]
  [3.22265354 2.02953975 2.67893565]]]
M2 = 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

Overlap between the ground state and the ansatz: 0.7645597592867793


In [100]:
@qml.qnode(dev)
def expected_value(params):
    ansatz(params)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

def loss_f(params):
    global beta
    return expected_value(params) + beta * swap_test(params)

In [76]:
cost_history_dict = {
    "iters": 0,
    "penalty": [],
    "overlap": [],
    "energy": []
}


# Define the vqd cost function
def vqd_cost_function(params):
    
    #print(cost_history_dict["iters"])
    cost_history_dict["iters"] += 1

    global prev_states
    energy = float(energy_expec(params))

    # Add penalty for overlap with previous states
    penalty = 0
    overlap = 0
    beta = 5

    for state in prev_states:
        overlap = abs(np.vdot(params.flatten(), state)) ** 2
        penalty += beta*overlap
    
    cost_history_dict["penalty"].append(penalty)
    cost_history_dict["overlap"].append(overlap)
    cost_history_dict["energy"].append(energy)

    return energy + penalty

In [101]:
vqd_start = datetime.now()

#variables
num_vqd_runs = 1
max_iterations = 1000
strategy = 'best1bin'
popsize = 15

#data arrays
energies = []
prev_states = []

for i in range(num_vqd_runs):

    run_start = datetime.now()

    if i % 1 == 0:
        print(f"Run: {i}")

    #Optimizer
    bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]

    # Differential Evolution optimization
    res = differential_evolution(loss_f, 
                                    bounds, 
                                    maxiter=max_iterations, 
                                    atol=tol,
                                    strategy=strategy, 
                                    popsize=popsize)
    
    print(res.success)
    energies.append(res.fun)
    prev_states.append(res.x) 

Run: 0
True


In [102]:
energies

[1.000000000000104]

In [103]:
prev_states

[array([2.77067906, 3.14159283, 3.01013479, 3.73077408, 4.96388377,
        0.65609386])]